In [ ]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function 
import argparse 
'''
python train.py --epochs 50 --batch-size 64 --save-dir weights 
We can see like this code with machine learning python script 
Many hyperparameters including training process would be setted with 
"python scrip_name.py --with some options" Thus we need to understand about argparse
'''
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
from torchvision import datasets, transforms 
from torch.autograd import Variable 


#Training settings 
batch_size = 64 

#MNIST Dataset 
train_dataset = datasets.MNIST(root = './data/',
                              train = True, 
                              transform = transforms.ToTensor(),
                              download=True)

test_dataset = datasets.MNIST(root= './data/',
                             train = False, 
                             transform = transforms.ToTensor()) 

#Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, 
                                          batch_size = batch_size, 
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = train_dataset, 
                                         batch_size = batch_size, 
                                         shuffle = False)

#Model generation  
class Net(nn.Module):
    
    def __init__(self): 
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20,kernel_size = 5)
        self.mp = nn.MaxPool2d(2) 
        self.fc = nn.Linear(320,10)
        
        #self.check_x_size = [] 
        #self.check_x_view = []
        
        '''
        torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, 
        padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        
        torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, 
        return_indices=False, ceil_mode=False)
        '''
        
    def forward(self,x):
        
        # addition for checking
        #self.check_x_size.append(x.size(0))
        
        in_size = x.size(0) # number of rows  
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size,-1) #flatten the tensor 
        
        '''
        x.size(0) 
         >> x.size(0) 은 2차원 행렬에서 row의 갯수를 반환. 
         >> x.size(1) 은 2차원 행렬에서 col의 갯수를 반환. 
         
         
        x.view(x.size(0),-1)  -> view is same with "reshape"
        
        tensor.view(a,b) 
         >> If you want to reshape this tensor to make it (a,b)
        
        x.view(x.size(0),-1)   
         >> number of row 는 x.size(0) 수 만큼 맞추고, cols는 앞의 조건에 맞춰 되는대로 많이 ,, 
        
        
        If there is any situation that you don't know how many rows you want but 
        are sure of the number of columns, then you can specify this with a -1. 
        (Note that you can extend this to tensors with more dimensions. Only one of the axis value can be -1).
        
        You have to flatten this to give it to the fully connected layer. 
        So you tell pytorch to reshape the tensor you obtained to have 
        specific number of columns and tell it to decide the number of rows by itself.
        
        '''
        
        x = self.fc(x) 
        return F.log_softmax(x)
    
    
model = Net()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5) 

def train(epoch): 
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader): 
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0 : 
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx*len(data), len(train_loader.dataset),
            100.*batch_idx / len(train_loader),loss.item()))

            
def test():
    model.eval()
    test_loss = 0 
    correct = 0 
    for data, target in test_loader : 
        data, target = Variable(data), Variable(target)
        output = model(data)
        #sum up batch loss 
        test_loss += F.nll_loss(output, target, size_average=False).data
        #get the index og the max log_probability 
        pred = output.data.max(1,keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set : Average loss : {:.4f}, Accuracy : {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100.* correct/len(test_loader.dataset)))
    
if __name__ == '__main__': 
    for epoch in range(1,10): 
        train(epoch)
        test()

In [2]:
t = torch.randn([2,3,5])
print(t)
print("t.size(0) : {}\tt.size(1): {}\tt.size(2): {}".format(t.size(0),t.size(1),t.size(2)))

tensor([[[-0.7797, -0.3800, -0.2669,  0.5075, -1.7320],
         [-0.7702,  0.3129, -1.7766,  0.2101,  1.1058],
         [-0.9809,  0.1280, -0.0468,  0.3478,  1.3486]],

        [[-0.0380, -0.7387, -1.2794,  0.3995, -0.9839],
         [-0.0183, -0.9246, -0.4916, -0.1553, -1.1914],
         [-0.3024,  0.0967, -0.7624,  0.2536,  2.8452]]])
t.size(0) : 2	t.size(1): 3	t.size(2): 5


주석참조주소<br> https://greeksharifa.github.io/pytorch/2018/11/10/pytorch-usage-03-How-to-Use-PyTorch/

argparse 및 pytorch model 

![aboout torch conv](kernel_size.jpg)